In [1]:
import pandas as pd
import json
import numpy as np

In [20]:
df = pd.read_csv("./rym_clean1.csv") # 5000 albums on Rate Your Music

# Search for Songs on the Spotify API

In [178]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import urllib.parse

# to run this, you'll need keys from the spotify API
auth_manager = SpotifyClientCredentials(
    client_id="",
    client_secret="",
)

In [179]:
sp = spotipy.Spotify(auth_manager=auth_manager)

In [54]:
spotify_uris = {}

In [95]:
for index, row in list(df.iterrows()):
    try:
        result = sp.search(urllib.parse.quote_plus(f"{row['release_name']} {row['artist_name']}")[:250], type="album", limit=1)
        if len(result["albums"]["items"]) > 0:
            spotify_uris[row["release_name"]] = result["albums"]["items"][0]["uri"]
    except:
        print(index)
        raise

In [102]:
result = sp.album_tracks(spotify_uris["To Pimp a Butterfly"])

In [165]:
track_names = {}

In [181]:
for index, (album, id) in enumerate(list(spotify_uris.items())[72:]):
    print(index)
    try:
        result = sp.album_tracks(id)
        if len(result["items"]) > 0:
            track_names[album] = [i["name"] for i in result["items"]]
    except:
        print(index, album, "needs special attention")
        raise

In [109]:
with open("./track-names.json", "w") as f:
  json.dump(track_names, f)

# Look up Song Audio features

In [9]:
feature_df1 = pd.read_csv("./SpotifyAudioFeaturesApril2019.csv")
feature_df2 = pd.read_csv("./top2018.csv")
feature_df3 = pd.read_csv("./Spotify-2000.csv")

In [10]:
feature_df2.rename(columns={"id": "track_id", "name": "track_name", "artists": "artist_name"}, inplace=True)
feature_df2

,track_id,track_name,artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6DCZcSspjsKoFjzjrWoCd,God's Plan,Drake,0.754,0.449,7.0,-9.211,1.0,0.1090,0.0332,0.000083,0.5520,0.3570,77.169,198973.0,4.0
1,3ee8Jmje8o58CHK66QrVC,SAD!,XXXTENTACION,0.740,0.613,8.0,-4.880,1.0,0.1450,0.2580,0.003720,0.1230,0.4730,75.023,166606.0,4.0
2,0e7ipj03S05BNilyu5bRz,rockstar (feat. 21 Savage),Post Malone,0.587,0.535,5.0,-6.090,0.0,0.0898,0.1170,0.000066,0.1310,0.1400,159.847,218147.0,4.0
3,3swc6WTsr7rl9DqQKQA55,Psycho (feat. Ty Dolla $ign),Post Malone,0.739,0.559,8.0,-8.011,1.0,0.1170,0.5800,0.000000,0.1120,0.4390,140.124,221440.0,4.0
4,2G7V7zsVDxg1yRsu7Ew9R,In My Feelings,Drake,0.835,0.626,1.0,-5.833,1.0,0.1250,0.0589,0.000060,0.3960,0.3500,91.030,217925.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,630sXRhIcfwr2e4RdNtjK,Rewrite The Stars,Zac Efron,0.684,0.619,10.0,-7.005,1.0,0.0386,0.0716,0.000000,0.1220,0.2840,125.046,217440.0,4.0
96,2xmrfQpmS2iJExTlklLoA,I Miss You (feat. Julia Michaels),Clean Bandit,0.638,0.658,3.0,-6.318,1.0,0.0456,0.2450,0.000004,0.0919,0.3300,105.076,205748.0,4.0
97,5WvAo7DNuPRmk4APhdPzi,No Brainer,DJ Khaled,0.552,0.760,0.0,-4.706,1.0,0.3420,0.0733,0.000000,0.0865,0.6390,135.702,260000.0,5.0
98,1j4kHkkpqZRBwE0A4CN4Y,Dusk Till Dawn - Radio Edit,ZAYN,0.258,0.437,11.0,-6.593,0.0,0.0390,0.1010,0.000001,0.1060,0.0967,180.043,239000.0,4.0


In [11]:
feature_df3.rename(columns=
    { 
        "Energy": "energy",
        "Danceability": "danceability",
        "Loudness(dB)": "loudness",
        "Liveness": "liveness",
        "Valence": "valence",
        "Length(Duration)": "duration_ms",
        "Acousticness": "acousticness",
        "Speechiness": "speechiness",
        "Title": "track_name",
        "Artist": "artist_name",
    },inplace=True
)

In [12]:
cols = [ "energy",
         "danceability",
         "liveness",
         "valence",
         "acousticness",
         "speechiness",
         "track_name",
         "artist_name",]

In [13]:
feature_df = pd.concat([feature_df1[cols], feature_df2[cols], feature_df3[cols]], ignore_index=True)

In [162]:
records = []
for row in feature_df.iterrows():
  title = row["track_name"]
  artist = row["artist_name"]
  

,energy,danceability,liveness,valence,acousticness,speechiness,track_name,artist_name
0,0.339,0.743,0.0812,0.118,0.005820,0.4090,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",YG
1,0.557,0.846,0.2860,0.371,0.024400,0.4570,BAND DRUM (feat. A$AP Rocky),YG
2,0.723,0.603,0.0824,0.382,0.025000,0.0454,Radio Silence,R3HAB
3,0.579,0.800,0.0994,0.641,0.029400,0.0701,Lactose,Chris Cooq
4,0.792,0.783,0.0332,0.928,0.000035,0.0661,Same - Original mix,Chris Cooq
...,...,...,...,...,...,...,...,...
132752,21.000,70.000,11.0000,72.000,84.000000,7.0000,Heartbreak Hotel,Elvis Presley
132753,76.000,36.000,76.0000,95.000,73.000000,6.0000,Hound Dog,Elvis Presley
132754,80.000,53.000,31.0000,97.000,74.000000,7.0000,Johnny B. Goode,Chuck Berry
132755,26.000,45.000,7.0000,60.000,54.000000,4.0000,Take Five,The Dave Brubeck Quartet


In [17]:
feature_df

,energy,danceability,liveness,valence,acousticness,speechiness,track_name,artist_name
0,0.339,0.743,0.0812,0.118,0.005820,0.4090,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",YG
1,0.557,0.846,0.2860,0.371,0.024400,0.4570,BAND DRUM (feat. A$AP Rocky),YG
2,0.723,0.603,0.0824,0.382,0.025000,0.0454,Radio Silence,R3HAB
3,0.579,0.800,0.0994,0.641,0.029400,0.0701,Lactose,Chris Cooq
4,0.792,0.783,0.0332,0.928,0.000035,0.0661,Same - Original mix,Chris Cooq
...,...,...,...,...,...,...,...,...
132752,21.000,70.000,11.0000,72.000,84.000000,7.0000,Heartbreak Hotel,Elvis Presley
132753,76.000,36.000,76.0000,95.000,73.000000,6.0000,Hound Dog,Elvis Presley
132754,80.000,53.000,31.0000,97.000,74.000000,7.0000,Johnny B. Goode,Chuck Berry
132755,26.000,45.000,7.0000,60.000,54.000000,4.0000,Take Five,The Dave Brubeck Quartet


In [18]:
# if you are recreating and don't want to use the spotify API
with open("./track-names.json") as f:
    track_names = json.load(f)

In [40]:
data = np.zeros((2251,9), dtype=object)
print(len(track_names))
in_rym = set()
j = 0
for i,album in enumerate(track_names):
    if i % 500 == 0:
        print(i)
    artist = df[df["release_name"] == album]["artist_name"].values[0]
    rating = df[df["release_name"] == album]["avg_rating"].values[0]
    for song in track_names[album]:
        search = feature_df[
            (feature_df["track_name"] == song) & (feature_df["artist_name"] == artist)
        ]
        if not search.empty:
            record = search.to_numpy()[0]
            in_rym.add(song)
            data[j,:8] = record
            data[j,8] = rating
            j += 1

4927
0
500
1000
1500
2000
2500
3000
3500
4000
4500


In [45]:
data = np.array(data)
cols = [
    "energy",
    "danceability",
    "liveness",
    "valence",
    "acousticness",
    "speechiness",
    "track_name",
    "artist_name",
    "rating"
]

In [46]:
training_df = pd.DataFrame(data, columns=cols)

In [50]:
training_df.to_csv("./rym-songs.csv", index=False)

In [34]:
other_df = pd.read_csv("./rym-songs.csv")

In [35]:
other_df

,Unnamed: 0,energy,danceability,liveness,valence,acousticness,speechiness,track_name,artist_name,in_rym
0,0,85.000,25.000,5.000,19.000,4.0000,6.0000,Paranoid Android,Radiohead,True
1,1,51.000,36.000,17.000,32.000,6.0000,3.0000,Karma Police,Radiohead,True
2,2,39.000,26.000,11.000,12.000,6.0000,3.0000,No Surprises,Radiohead,True
3,3,48.000,36.000,18.000,36.000,52.0000,8.0000,Time,Pink Floyd,True
4,4,22.000,28.000,8.000,18.000,77.0000,3.0000,The Great Gig in the Sky,Pink Floyd,True
...,...,...,...,...,...,...,...,...,...,...
2246,2246,0.752,0.489,0.328,0.438,0.0358,0.2610,Stylish Spit,The Garden,True
2247,2247,0.828,0.694,0.147,0.696,0.0145,0.0349,No Destination,The Garden,True
2248,2248,69.000,69.000,20.000,93.000,17.0000,3.0000,Take A Chance On Me,ABBA,True
2249,2249,56.000,66.000,6.000,62.000,40.0000,3.0000,The Name Of The Game,ABBA,True


In [53]:
non_rym_songs = feature_df[~feature_df["track_name"].isin(in_rym)].sample(2251)

In [54]:
non_rym_songs["rating"] = -1

In [56]:
non_rym_songs.to_csv("non-rym-songs.csv", index=False)

In [57]:
feature_df[~feature_df["track_name"].isin(in_rym)]

,energy,danceability,liveness,valence,acousticness,speechiness,track_name,artist_name
0,0.339,0.743,0.0812,0.118,0.005820,0.4090,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",YG
1,0.557,0.846,0.2860,0.371,0.024400,0.4570,BAND DRUM (feat. A$AP Rocky),YG
2,0.723,0.603,0.0824,0.382,0.025000,0.0454,Radio Silence,R3HAB
3,0.579,0.800,0.0994,0.641,0.029400,0.0701,Lactose,Chris Cooq
4,0.792,0.783,0.0332,0.928,0.000035,0.0661,Same - Original mix,Chris Cooq
...,...,...,...,...,...,...,...,...
132751,7.000,17.000,14.0000,10.000,92.000000,3.0000,Summertime,Louis Armstrong
132752,21.000,70.000,11.0000,72.000,84.000000,7.0000,Heartbreak Hotel,Elvis Presley
132753,76.000,36.000,76.0000,95.000,73.000000,6.0000,Hound Dog,Elvis Presley
132754,80.000,53.000,31.0000,97.000,74.000000,7.0000,Johnny B. Goode,Chuck Berry
